In [33]:
!pip3 install cryptocompare

In [1]:
import cryptocompare
import requests
import json
import pandas as pd
import os
from time import sleep
from datetime import datetime
from dotenv import load_dotenv

In [2]:
load_dotenv()
apikey = os.getenv('apikey')
cryptocompare_api = os.getenv('cryptocompare_api')

In [3]:
def getPublicKeyTransactions():
    userInput = input('Choose profile:\n'
                  '1. Binance\n'
                  '2. Uniswap\n'
                  '3. Gemini\n')
    
    def getAddress(arg):
        addressBook = {1:'0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE',
                       2:'0x88599C44e8EeB2Bbc2C1EB4749D37E09c8282d55',
                       3:'0xd24400ae8BfEBb18cA49Be86258a3C749cf46853'}
        return addressBook.get(arg)
              
    address = getAddress(int(userInput))
    
    getAddressTransactions = f'https://api.ethplorer.io/getAddressTransactions/{address}?limit=1000&apiKey={apikey}'
    getAddressTransactionsResponse = requests.get(getAddressTransactions).json()
    
    timestamp = []
    converted_timestamp = []
    hash_num = []
    value = []
    from_add = []
    to_add = []
    
    for transaction in getAddressTransactionsResponse:
        timestamp.append(transaction['timestamp'])
        converted_timestamp.append(datetime.fromtimestamp(transaction['timestamp']))
        hash_num.append(transaction['hash'])
        value.append(transaction['value'])
        from_add.append(transaction['from'])
        to_add.append(transaction['to'])

    df = pd.DataFrame({'converted_timestamp':converted_timestamp})
    ts = df['converted_timestamp']

    def roundNearestMinute(ts):
        return ts.dt.round('1min')

    rounded_timestamp = roundNearestMinute(ts)
    df = pd.DataFrame({'timestamp': timestamp, 'compare_timestamp': rounded_timestamp, 'transaction_hash': hash_num, 'value_in_eth': value, 'from_address': from_add, 'to_address': to_add})

    return df
    
pk_df = getPublicKeyTransactions()

Choose profile:
1. Binance
2. Uniswap
3. Gemini
 1


In [4]:
def getHistoricalPrice():
    set_crypto_key = f'cryptocompare.cryptocompare._set_api_key_parameter{cryptocompare_api}'
    getHistETHPrice = cryptocompare.get_historical_price_minute('ETH', currency='USD', limit=1440)

    timestamp = []
    converted_timestamp = []
    close = []

    for time in getHistETHPrice:
        timestamp.append(time['time'])
        converted_timestamp.append(datetime.fromtimestamp(time['time']))
        close.append(time['close'])

    df = pd.DataFrame({'timestamp':timestamp, 'compare_timestamp':converted_timestamp, 'close':close}).sort_values(by=['compare_timestamp'], ascending=False)
    return df

hp_df = getHistoricalPrice()

In [5]:
final_df = pd.merge(pk_df, hp_df, on='compare_timestamp')#.sort_values(by=['compare_timestamp'], ascending=True)
final_df.to_csv('getTransactionData.csv')
final_df.head(25)

,timestamp_x,compare_timestamp,transaction_hash,value_in_eth,from_address,to_address,timestamp_y,close
0,1620741149,2021-05-11 09:52:00,0xdbf010d8088f6ee0d6d27079b759e5106f66b384521e...,0.029720,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xd66c405fd806514b4ee9a61edfb43d19d11b664d,1620741120,3958.98
1,1620741104,2021-05-11 09:52:00,0x60b7c3084204091a5b345acaa230480fc015168b09c3...,0.031320,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x9610cc904edec2790d0c82ee53e7796842b5722a,1620741120,3958.98
2,1620741104,2021-05-11 09:52:00,0xade446bfdbc508eb61e35c2e0d9bf8511a8ad6e012be...,0.061580,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x3dda00a5350c939d029ba8fbd7cd63baae24a08a,1620741120,3958.98
3,1620740988,2021-05-11 09:50:00,0xc588c290740ba9851a22d3f966bb1ea22ce768ea4ab1...,28.000000,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x8ff8e9c72281e77c158fe39beb9efbc4855f1395,1620741000,3955.24
4,1620740892,2021-05-11 09:48:00,0x4909f63ae74f04ad39c8ed81f4097ad321134f328caf...,0.025416,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x49eb1ead56c8d0ca2a1a8f28780567a347eb9382,1620740880,3948.57
5,1620740892,2021-05-11 09:48:00,0x0eabbe59def1e4d04c526e9cb75f1d21e67a5fe19139...,0.200000,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xbcf2c1892de06e8659ea5d4283fbc8fd228fff22,1620740880,3948.57
6,1620740892,2021-05-11 09:48:00,0x8ef80586ba350b5cfc516c8f90ef9bc250b349be55a0...,0.067683,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xd86fe4b67515210d91c4a41800527e0bbe233822,1620740880,3948.57
7,1620740892,2021-05-11 09:48:00,0xe96fa15568c1ba8c102465b3298af4a53f42de42e07d...,0.122990,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x5be8fa61e1d113739d90d9d9fd4979b1d50d254b,1620740880,3948.57
8,1620740892,2021-05-11 09:48:00,0x678c3e7e43205c80381db8ea010030733bcd317093f8...,0.041450,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xac8d1e89ca4a081afb1be5a48b2318dd549f8e3d,1620740880,3948.57
9,1620740892,2021-05-11 09:48:00,0xe1c491f420b422ef03fb835bd5770482f640ded303d0...,0.088881,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xd7335a7f2620c3dad3b16d4f4373363dcd1ea2bc,1620740880,3948.57
